In [50]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_groq import ChatGroq

# Load the GROQ API key from environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Initialize the ChatGroq instance
llm = ChatGroq(temperature=0.5, groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")

In [ ]:
# json_schema = {
#     "title": "joke",
#     "description": "Joke to tell user.",
#     "type": "object",
#     "properties": {
#         "setup": {
#             "type": "string",
#             "description": "The setup of the joke",
#         },
#         "punchline": {
#             "type": "string",
#             "description": "The punchline to the joke",
#         },
#         "rating": {
#             "type": "integer",
#             "description": "How funny the joke is, from 1 to 10",
#             "default": None,
#         },
#     },
#     "required": ["setup", "punchline"],
# }
# structured_llm = llm.with_structured_output(json_schema)

# structured_llm.invoke("Tell me a joke about cats")

{'setup': 'Why did the cat join a band?',
 'punchline': 'Because it wanted to be the purr-cussionist!',
 'rating': 8}

In [ ]:
json_schema = {
    "title": "mcq quiz generator",
    "description": "Gives the quiz questions and correct answer and 4 options for each question and a hint for each question",
    "type": "object",
    "properties": {
        "question": {
            "type": "string",
            "description": "Give the quiz question",
        },
        "all 4 options": {
            "type": "dict",
            "items" : {"type": "string"},
            "description": "All the 4 options for the question as a) b) c) d). there should be only 4 options",
            "default": None,
        },

        "Correct answer": {
            "type": "string",
            "description": "The correct answer to the question along with like a) b) c) d) whichever is correct",
        },
        
        "hint": {
            "type": "string",
            "description": "A hint for the question",
            "default": None,
        },
        
    },
    "required": ["question", "Correct answer", "all 4 options", "hint"],
}
structured_llm = llm.with_structured_output(json_schema)

prompt = "Generate a multiple choice quiz question about the capital of France"
structured_llm.invoke(prompt)

{'name': 'Python',
 'question': 'What is the use of the pass statement in Python?',
 'Correct answer': 'pass is used as a placeholder when a statement is required syntactically but no execution of code is necessary.',
 'all 4 options': {'a': 'To skip the execution of a block of code',
  'b': 'To give a placeholder when a statement is required syntactically but no execution of code is necessary',
  'c': 'To stop the execution of a loop',
  'd': 'To start a new function'},
 'hint': 'It is often used in a class definition when no methods are defined.'}

In [76]:
from langchain_core.prompts import PromptTemplate

chat_history = []
json_schema = {
    "title": "mcq quiz generator",
    "description": "Gives the quiz questions and correct answer and 4 options for each question and a hint for each question",
    "type": "object",
    "properties": {
        "question": {
            "type": "string",
            "description": "Give the quiz question"
        },
        "all_4_options": {
            "type": "dict",
            "description": "All the 4 options for the question as a) b) c) d). There should be only 4 options",
            "properties": {
                "a": {"type": "string"},
                "b": {"type": "string"},
                "c": {"type": "string"},
                "d": {"type": "string"}
            }
        },
        "correct_answer": {
            "type": "string",
            "description": "The correct answer to the question along with like a) b) c) d) whichever is correct"
        },
        "hint": {
            "type": "string",
            "description": "A hint for the question",
            "default": None
        }
    },
    "required": ["question", "correct_answer", "all_4_options", "hint"]
}

structured_llm = llm.with_structured_output(json_schema)
prompt_template = PromptTemplate.from_template("give me a quiz on topic- {topic}. Dont give the same question twice")

prompt = prompt_template.invoke({"topic": "python"})

response = structured_llm.invoke(prompt)
response

{'name': 'Python',
 'question': "What is the primary purpose of the 'pass' statement in Python?",
 'correct_answer': 'The pass statement is a placeholder when a statement is required syntactically but no execution of code is necessary.',
 'all_4_options': {'a': 'To create a loop that runs indefinitely',
  'b': 'To create a placeholder when a statement is required syntactically but no execution of code is necessary',
  'c': 'To skip to the next iteration of a loop',
  'd': 'To end a loop prematurely'},
 'hint': "It's used when you want to do nothing"}

In [77]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

# Load the GROQ API key from environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Initialize the ChatGroq instance
llm = ChatGroq(temperature=0.5, groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")

# Define the JSON schema
json_schema = {
    "title": "mcq quiz generator",
    "description": "Gives the quiz questions and correct answer and 4 options for each question and a hint for each question",
    "type": "object",
    "properties": {
        "question": {
            "type": "string",
            "description": "Give the quiz question"
        },
        "all_4_options": {
            "type": "object",
            "description": "All the 4 options for the question as a) b) c) d). There should be only 4 options",
            "properties": {
                "a": {"type": "string"},
                "b": {"type": "string"},
                "c": {"type": "string"},
                "d": {"type": "string"}
            }
        },
        "correct_answer": {
            "type": "string",
            "description": "The correct answer to the question along with like a) b) c) d) whichever is correct"
        },
        "hint": {
            "type": "string",
            "description": "A hint for the question",
            "default": None
        }
    },
    "required": ["question", "correct_answer", "all_4_options", "hint"]
}

# Chain the LLM to the structured output
structured_llm = llm.with_structured_output(json_schema)

# Create a prompt template
prompt_template = PromptTemplate.from_template("Give me a quiz question on the topic of {topic}. Ensure each question is unique.")

# Generate 5 different quiz questions
quiz_questions = []
for _ in range(5):
    prompt = prompt_template.invoke({"topic": "Python"})
    response = structured_llm.invoke(prompt)
    quiz_questions.append(response)

# Print the quiz questions
for i, quiz in enumerate(quiz_questions, 1):
    print(f"Quiz Question {i}:")
    print(quiz)
    print("\n")

Quiz Question 1:
{'name': 'Python', 'question': "What is the use of the 'break' statement in Python?", 'correct_answer': 'The break statement is used to terminate the loop prematurely, i.e., to exit the loop when a particular condition is met.', 'hint': 'It is used in loops such as for and while', 'all_4_options': {'a': 'To create a new loop', 'b': 'To exit the loop prematurely', 'c': 'To skip an iteration of the loop', 'd': 'To repeat a block of code'}}


Quiz Question 2:
{'question': "What does the 'pass' statement do in Python?", 'correct_answer': 'pass', 'all_4_options': {'a': 'It is used to skip the rest of the current iteration in a loop', 'b': 'It is used to skip the rest of the current loop', 'c': 'It is used to do nothing in a function', 'd': 'It is used to terminate a loop immediately'}, 'hint': 'It is often used as a placeholder when a statement is required syntactically but no execution of code is necessary'}


Quiz Question 3:
{'name': 'Python', 'question': "What is the pu

In [87]:
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

# Load the GROQ API key from environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Initialize the ChatGroq instance
llm = ChatGroq(temperature=0.5, groq_api_key=GROQ_API_KEY, model_name="llama-3.1-8b-instant")

# Define the JSON schema
json_schema = {
    "title": "mcq quiz generator",
    "description": "Gives the quiz questions and correct answer and 4 options for each question and a hint for each question",
    "type": "object",
    "properties": {
        "question": {
            "type": "string",
            "description": "Give the quiz question"
        },
        "all_4_options": {
            "type": "object",
            "description": "All the 4 options for the question as a) b) c) d). There should be only 4 options",
            "properties": {
                "a": {"type": "string"},
                "b": {"type": "string"},
                "c": {"type": "string"},
                "d": {"type": "string"}
            }
        },
        "correct_answer": {
            "type": "string",
            "description": "The correct answer to the question along with like a) b) c) d) whichever is correct"
        },
        "hint": {
            "type": "string",
            "description": "A hint for the question",
            "default": None
        }
    },
    "required": ["question", "correct_answer", "all_4_options", "hint"]
}

# Chain the LLM to the structured output
structured_llm = llm.with_structured_output(json_schema)

# Create a prompt template
prompt_template = PromptTemplate.from_template("""
You are a quiz generator. Generate a multiple-choice question (MCQ) on the topic of {topic}.
Provide the question, four options labeled a, b, c, and d, the correct answer along with the correct option, and a hint.
Ensure there are exactly four options.
THE FORMAT SHOULD BE LIKE THIS STRICTLY like the one specified in the schema.
Here are the questions that have already been generated:
{history}
So these questions or related questions should not be generated again. Every question should be unique.
""")

# Generate 5 different quiz questions
quiz_questions = []
history = ""

for _ in range(5):
    prompt = prompt_template.invoke({"topic": "Python", "history": history})
    response = structured_llm.invoke(prompt)
    quiz_questions.append(response)
    history += f"\nQuestion: {response['question']}\nOptions: {response['all_4_options']}\n"


print(quiz_questions)

[{'name': 'Python', 'question': "What is the purpose of the 'pass' statement in Python?", 'correct_answer': 'The correct answer is a) It is used as a placeholder when a statement is required syntactically but no execution of code is necessary.', 'all_4_options': {'a': 'It is used as a placeholder when a statement is required syntactically but no execution of code is necessary.', 'b': 'It is used to stop the execution of a program.', 'c': 'It is used to continue the execution of a program.', 'd': 'It is used to skip a section of code.'}, 'hint': 'This statement is often used in situations where a statement is required syntactically, but no execution of code is necessary.'}, {'name': 'Python', 'question': "What is the purpose of the 'else' block in a for loop in Python?", 'correct_answer': 'It is used when a specific condition is met after the loop has finished iterating over all the items.', 'all_4_options': {'a': 'It is used when a specific condition is met after the loop has finished 